In [1]:
import nltk
# nltk.download("stopwords")

In [3]:
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as functions

In [4]:
spark = SparkSession.builder.master("local[*]").appName("WordCloud").getOrCreate()

23/01/28 21:14:03 WARN Utils: Your hostname, FLNNOTFS033753 resolves to a loopback address: 127.0.1.1; using 192.168.15.12 instead (on interface enp1s0)
23/01/28 21:14:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/28 21:14:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
